In [1]:
# Make sure helpers functionality can be imported
import os
import sys

project_path, _ = os.path.split(os.getcwd())
if project_path not in sys.path:
    sys.path.insert(0, project_path)

In [10]:
# Dependencies
# pip install numpy
# pip install pandas
# pip install sklearn

# Ignore warnings
import warnings; warnings.simplefilter("ignore")

# Import libraries
import numpy as np
from helpers.utils.logger import Logger

# Set the random generator seed
seed = 42

## Load an experimental data

In [11]:
# Load an example dataset
from sklearn.datasets import load_breast_cancer

dataset = load_breast_cancer()
X = dataset.data
y = dataset.target

## Cross-validate an example classifier

In [12]:
from helpers.classification.validation import cross_validate_classifier
from sklearn.linear_model import LogisticRegression

# Initialize the classifier
classifier = LogisticRegression(random_state=seed, solver="lbfgs")

# Define the classification options
threshold = 0.5
metrics = ("acc", "sen", "spe")
num_folds = 10
num_repetitions = 20
verbose = False

# Cross-validate the classifier
results = cross_validate_classifier(X, 
                                    y, 
                                    classifier, 
                                    threshold=threshold, 
                                    metrics=metrics, 
                                    num_folds=num_folds, 
                                    num_repetitions=num_repetitions, 
                                    seed=seed)

print("-------------------------")
print("Cross-validation results:")
print("-------------------------")
print("")

for metric in metrics:
    print("{} = {:.2f} +- {:.2f}".format(metric, float(np.mean(results[metric])), float(np.std(results[metric]))))

-------------------------
Cross-validation results:
-------------------------

acc = 0.94 +- 0.02
sen = 0.96 +- 0.04
spe = 0.91 +- 0.07
